In [1]:
import platform
import folium
import math
from tqdm import tqdm

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action = 'ignore')

from geoband.API import *
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

if platform.system() == 'Darwin': # 맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': # 윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': # 리눅스 (구글 Colab)
        plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## 전처리한 weather 데이터 불러오기

In [2]:
accident_list = pd.read_csv('1.대전광역시_교통사고내역(2017~2019).csv')

In [3]:
weather_arrange = pd.read_csv('weather_arrange.csv')
weather_arrange.columns = ['사고일', '평균온도', '최저온도', '최고온도', '최대풍속' ,'평균풍속', '평균습도', '평균지면온도', '강수량' ,'적설량', '안개시간']

print(np.shape(weather_arrange))
weather_arrange.head(3)

(1095, 11)


,사고일,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
0,2017-01-01,0,0,0,0,1.0,0,0,0,0,0
1,2017-01-02,0,0,0,0,0.0,0,0,0,0,0
2,2017-01-03,0,0,0,0,0.0,0,0,0,0,0


In [4]:
acci = accident_list.merge(weather_arrange, on='사고일', how='left')
acci_sum = acci.groupby('gid').sum(['평균온도', '최고온도', '최저온도', '최대풍속', '평균풍속', '평균습도','평균지면온도', '강수량', '적설량', '안개시간']) 
acci_sum.reset_index(inplace=True)
acci_sum = acci_sum[['gid', '평균온도', '최저온도', '최고온도', '최대풍속', '평균풍속', '평균습도', '평균지면온도', '강수량', '적설량', '안개시간']]
print(np.shape(acci_sum))
acci_sum.head(3)

(6068, 11)


,gid,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
0,다바780093,0,0,0,0,1.0,0,0,0,0,0
1,다바781090,0,0,0,0,0.0,0,0,0,0,1
2,다바781091,0,0,0,0,0.0,0,0,0,0,0


## 12. 전처리한 weather 데이터와 현재까지 구축한 데이터와 병합

In [5]:
accident_count_filter_5 = pd.read_csv('accident_count_filter_5.csv', index_col = 0)
accident_count_filter_6 = accident_count_filter_5.merge(acci_sum, on='gid', how='left')
accident_count_filter_6 = accident_count_filter_6.fillna(0)
print(np.shape(accident_count_filter_6))
accident_count_filter_6.head(2)

(5556, 23)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,...,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
0,다바931203,2,MULTIPOLYGON (((127.4230710131166 36.380134552...,2,2,127.423628,36.380586,1.0,3.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,다바861174,0,MULTIPOLYGON (((127.3450791441312 36.353914265...,0,0,127.345636,36.354366,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
accident_count_filter_6.to_csv('accident_count_filter_6.csv')
# 신호등(보행등), 신호등(차량등), cctv수, 혼잡빈도, 혼잡시간, 교통추정량, 날씨 데이터까지 merge한 상태.